### Add model, connection and device to settings:

In [4]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings
from ddeint import ddeint

# пути и названия файлов
modelFileName = 'problems/Circulating_red_blood_cells'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


In [5]:
print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)

equations:
[["U'=1/(1+(U(t-1))^8)-b*U"]]
params:
['a', 'b']
paramValues:
[{'a': 1, 'b': 0.5745}, {'a': 1, 'b': 0.2}, {'a': 1, 'b': 0.01}]


### Run solver:

In [6]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from hybriddomain.solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc
progress = ProgressNotebook(100, prefix='solving')

progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(modelFileName)

from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(settings.paths["hd"]["json"])
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving,remove_old=False)
        
        # result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
    display(Javascript("alert('Done');"))
    
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)

#remoteProjectRun(settings, 1, progress)
display(bTest)

IntProgress(value=0, description='param: ', max=2)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run Circulating_red_blood_cells ', style=ButtonStyle())

### Results:

In [7]:
model.readResults(result_format=1)
# available result files:
print("result files: ", model.results_paths)
# available plot files:
print("plot files: ", model.plots_paths)

result files:  {'u': ['problems/Circulating_red_blood_cells/out/u_seq0.out', 'problems/Circulating_red_blood_cells/out/u_seq1.out', 'problems/Circulating_red_blood_cells/out/u_seq2.out']}
plot files:  {}


In [8]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['u'])


### Video:

In [9]:
# available plots:
model.plots_paths

{}

In [10]:
display(HTML(model.get_video('U')))

KeyError: 'U'

### U(x, t-fixed),V(x, t-fixed):

In [11]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']

print(len(result))
def f(t):
    result_x = np.array([result[time][0][t] for time in times])
    result_abs = np.array([math.sin(t/(len(result[0][0])-1)*3.1415926)*math.exp(-time)  for time in times])
    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(time,result_x)
    # точное решение
    plt.plot(time, result_abs)
w = interactive(f,t=(0,len(result[0][0])-1,1))
display(w)

plt.show()

101


interactive(children=(IntSlider(value=157, description='t', max=314), Output()), _dom_classes=('widget-interac…

In [12]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']


def f(t):
    result_x = np.array([result[time][0][t] for time in times])
    result_abs = np.array([math.sin(t/(len(result[0][0])-1)*3.1415926)*math.exp(-time)  for time in times])
    
    result_raz=np.array([abs(result[time][0][t]-math.sin(t/(len(result[0][0])-1)*3.1415926)*math.exp(-time)) for time in times])
    #сравнивание с погрешностью
    plt.plot(time,result_raz)
w = interactive(f,t=(0,len(result[0][0])-1,1))
display(w)

plt.show()

interactive(children=(IntSlider(value=157, description='t', max=314), Output()), _dom_classes=('widget-interac…

In [13]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']


def f(t):
    result_x = np.array([result[time][0][t] for time in times])
    result_abs = np.array([math.sin(t/(len(result[0][0])-1)*3.1415926)*math.exp(-time)  for time in times])
    
    result_raz=np.array([abs(result[time][0][t]-math.sin(t/(len(result[0][0])-1)*3.1415926)*math.exp(-time))/abs(result[time][0][t]) for time in times])
    #сравнивание абсолютной погрешности
    plt.plot(time,result_raz)
w = interactive(f,t=(0,len(result[0][0])-1,1))
display(w)

plt.show()

interactive(children=(IntSlider(value=157, description='t', max=314), Output()), _dom_classes=('widget-interac…